In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error

2022-09-26 11:32:08.818610: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 11:32:08.988242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-26 11:32:08.988270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-26 11:32:09.015336: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 11:32:09.691932: W tensorflow/stream_executor/platform/de

In [3]:
new_braunfels_data_path = os.path.join("..","input_data","array_input","New_Braunfels.npy")
del_rio_data_path = os.path.join("..","input_data","array_input","Del_rio_data.npy")

In [4]:
del_rio_history = np.load(del_rio_data_path)
New_Braunfels_history = np.load(new_braunfels_data_path)

In [5]:
print(New_Braunfels_history.shape)

(1, 138, 64, 64)


In [32]:
def make_time_arr(history):
    time_arr = []
    for j in range(64):
        for i in range(64):
            arr = []
            for l in range(history.shape[1]):
                arr.append(history[0][l][j][i])
            time_arr.append(arr)
    return time_arr

In [33]:
new_braunfels_time_arr = make_time_arr(New_Braunfels_history)
del_rio__time_arr = make_time_arr(del_rio_history)

In [34]:
print(len(new_braunfels_time_arr[0]))
print(len(del_rio__time_arr[0]))

138
186


In [35]:
new_braunfels_data = np.array(new_braunfels_time_arr)
del_rio_data = np.array(del_rio__time_arr)
print(new_braunfels_data.shape)
print(del_rio_data.shape)

(4096, 138)
(4096, 186)


In [36]:
def LSTM_model(data):
    for i in range(0,len(data),32):
        batch = data[i:i+32]
        batch_reshaped = np.reshape(batch, (batch.shape[0], batch.shape[1], 1))
        lstm = tf.keras.layers.LSTM(10)
        output = lstm(batch_reshaped)
        
        dense_nn = tf.keras.layers.Dense(1)
        output = dense_nn(output)
        if i==0:
            prediction = np.array(output)
        else:
            prediction = np.concatenate((prediction,output), axis = 0)
    print(prediction.shape)
    return prediction

In [40]:
new_braunfels_prediction = LSTM_model(new_braunfels_data[:,:-1])
del_rio_prediction = LSTM_model(del_rio_data[:,:-1])

(4096, 1)
(4096, 1)


In [44]:
def mse(A,B):
    mse = mean_squared_error(A, B)
    return mse

In [104]:
print(new_braunfels_data[:,-1])
print(new_braunfels_prediction)

[0. 0. 0. ... 0. 0. 0.]
[[-0.00063613]
 [-0.00077811]
 [-0.00078349]
 ...
 [-0.00056113]
 [-0.00019311]
 [ 0.00131933]]


In [45]:
new_braunfels_mse = mse(new_braunfels_data[:,-1], new_braunfels_prediction)
del_rio_mse = mse(del_rio_data[:,-1], del_rio_prediction)
print(new_braunfels_mse, del_rio_mse)

0.0028888786222031806 0.0005028951479690945


In [38]:
# new_braunfels_prediction = np.reshape(new_braunfels_prediction,(64,64))
# del_rio_prediction = np.reshape(del_rio_prediction,(64,64))

Spatial Evaulation

In [98]:
def create_model():
  model = tf.keras.Sequential([
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dense(1, activation='softmax')
  ])

  model.compile(optimizer="Adam", loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalCrossentropy()])

  return model

In [99]:
model = create_model()

out = model(np.reshape(new_braunfels_data[:3000,:-1], (new_braunfels_data[:3000,:-1].shape[0], new_braunfels_data[:3000,:-1].shape[1], 1)))

In [93]:
y_true.shape, out.shape

((3000,), TensorShape([3000, 1]))

In [105]:
tf.losses.mse(y_true.reshape(-1, 1), out)

<tf.Tensor: shape=(3000,), dtype=float32, numpy=
array([0.2491291 , 0.24912259, 0.24938166, ..., 0.24480188, 0.24533348,
       0.24399492], dtype=float32)>

In [101]:
tf.losses.categorical_crossentropy(y_true.reshape(-1, 1), out)

<tf.Tensor: shape=(3000,), dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>

In [90]:
out

<tf.Tensor: shape=(3000, 1), dtype=float32, numpy=
array([[0.99999994],
       [0.99999994],
       [0.99999994],
       ...,
       [0.99999994],
       [0.99999994],
       [0.99999994]], dtype=float32)>

In [86]:
y_true = new_braunfels_data[:3000,-1]
y_true.shape

(3000,)

In [87]:
y_true_ones = np.ones(3000, )

In [88]:
checkpoint_path = "training_1/cp.ckpt"

lstm_model = create_model()
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
batch_reshaped = np.reshape(new_braunfels_data[:3000,:-1], (new_braunfels_data[:3000,:-1].shape[0], new_braunfels_data[:3000,:-1].shape[1], 1))
print(batch_reshaped.shape)
lstm_model.fit(batch_reshaped, y_true_ones,callbacks=[cp_callback])

(3000, 137, 1)
93/94 [============================>.] - ETA: 0s - loss: 0.0000e+00 - categorical_crossentropy: 0.0000e+00
Epoch 1: saving model to training_1/cp.ckpt
94/94 [==============================] - 5s 30ms/step - loss: 0.0000e+00 - categorical_crossentropy: 0.0000e+00


In [69]:
loss, acc = lstm_model.evaluate(batch_reshaped, new_braunfels_data[:3000,-1], verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

94/94 - 1s - loss: 1.5418e-04 - mae: 0.0091 - 1s/epoch - 11ms/step
Untrained model, accuracy:  0.91%


In [70]:
lstm_validation_model = create_model()
lstm_validation_model.load_weights(checkpoint_path)

batch_reshaped = np.reshape(new_braunfels_data[3000:,:-1], (new_braunfels_data[3000:,:-1].shape[0], new_braunfels_data[3000:,:-1].shape[1], 1))

loss, acc = lstm_validation_model.evaluate(batch_reshaped, new_braunfels_data[3000:,-1], verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

35/35 - 1s - loss: 3.1826e-04 - mae: 0.0137 - 945ms/epoch - 27ms/step
Untrained model, accuracy:  1.37%
